# Example using CBLAST with hemibrain neurons

This example shows how to cluster a set of neurons.  The clustering is meant to be somewhat iterative.
After clustering the data and spot checking the results, the types can be fed back into the algoritm and used for debugging.  Alternatively, the clustering can be used to evaluate previously clustered or categorized data.

Ground truth data is provided for this example.

Note: If cell types are already annotated in neuprint and one is just trying to better assess or slightly revise the current clustering, one should just generate features with "cblast.features.compute_connection_similarity_features(client, dataset, neuronlist)"

In [45]:
import pandas as pd

sample data of neuron body ids and their corresponding types

In [46]:
# sample data

neuronlist=[633546217, 416642425, 634962055, 5813027103, 788794171, 694920753, 948709216,
         1011447819, 918334668, 949710555, 917647959, 919763043, 1036637638, 858587718,
         1228692168, 1198330641, 1730608428, 5813060726, 541127846, 978733459, 1043825714]
typelist=['EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'ExR1', 'ExR1', 'ExR1', 'ExR1', 'ExR1',
       'ExR3', 'ExR3', 'ExR4', 'ExR4', 'ExR4', 'ExR4', 'ExR5', 'ExR5', 'ExR5', 'ExR5']

# create ground truth map
gtmap = pd.DataFrame.from_dict({"bodyid": neuronlist, "type": typelist})

Import neuprint python library and cblast and setup connection to the hemibrain dataset

**Note: set NEUPRINT_APPLICATION_CREDENTIALS=YOURTOKEN**

In [47]:
import neuprint as neu
import cblast
client = neu.Client("neuprint-test.janelia.org")
dataset = "hemibrain:v1.0"

Generate features for the list of neurons using the cblast workflow (seeded with region projection features)

The user must specify the average size of each cluster that is expected (e.g., 4 neurons per cluster).  For production, one should set *cluster_neighbors=True*.  This will take a lot longer to run but will give a better result.  One can also set the neighbor expected cluster size (*neighbor_npc*).

In [48]:
# generate features -- enable cluster_neighbors when running in production -- will take a lot longer to run
features = cblast.recipe.cblast_workflow_simple(client, dataset, neuronlist, 4) #, cluster_neighbors=True, neighbor_npc=10)

fetch batch 0
Computed initial clusters
fetch batch 0
Computed type clusters: 0.8211274758897206
Computed type clusters: 0.8209197223143072


* generate a cluster tree
* estimate a number of partitions

In [49]:
hcluster = cblast.cluster.hierarchical_cluster(features)
cluster, testmap = hcluster.get_partitions(5) # choose 5 partitions

Visualize the clusters using umap feature reduction.  For this example, the ground truth is used to provide a visual baseline.

**Note: one can see an ExR1 neuron that does not cluster very well.  This might indicate an error in the ground truth.**

In [50]:
cblast.view.features_scatterplot2D(features, clusters=testmap, groundtruth=gtmap)

Loading BokehJS ...

The "testmap" results can be modified and fed back into the connectivity-based feature generator.  For this example, we will just feed the ground truth mappings.

In [51]:
features2 = cblast.features.compute_connection_similarity_features(client, dataset, neuronlist, customtypes=gtmap)
hcluster2 = cblast.cluster.hierarchical_cluster(features2)
cluster2, testmap2 = hcluster.get_partitions(5)
cblast.view.features_scatterplot2D(features, clusters=testmap, groundtruth=gtmap)

fetch batch 0


Loading BokehJS ...

After this, theree is still an outlier neuron 1011447819 (ExR1) that should be similar to 917647959 (ExR1)

One can visualize the biggest differences and similarities between them

**Note: debugging would not be as easy for the features computed from the recipe since feature names could be based on temporary cell type ids**

In [52]:
# choose most impact features for two bodies that have the same type name but do not cluster similarly
filtered_features = cblast.features.find_max_differences(features2, 1011447819, 917647959, numfeatures=4)

# visualize greatest similarities and differences
cblast.view.features_compareheatmap(filtered_features)

:HeatMap   [feature,bodyid]   (score)

Based on the above results, the outlier neuron is much smaller than an other ExR1.  There is also a connection to ExR1 from the outlier that does not appear to exist in the other example.

Another way to examine the data (beyond just looking at the clustered data and inspecting salient feature differences), is to find the closest match for each neuron in the list.

In [53]:
# for each neuron, find the closest neuron to it
matches = cblast.cluster.get_strongest_matches(features2)

# with ground truth, one can evaluate the agreement rate
rate, exceptions = cblast.cluster.agreement_rate(matches, gtmap)
print("Percentage of neurons whose closest match is the same cell type using th ground truth:", rate)
print("List of groupings that disagree with ground truth", exceptions)

Percentage of neurons whose closest match is the same cell type using th ground truth: 95.23809523809523
List of groupings that disagree with ground truth [(1011447819, 'ExR1', 788794171, 'EPG')]


Given two clustering results, one can examine the differences between them.  This example compares clusters produced by simple ROI overlap (features_comp) with the initial iterative features generated.

In [54]:
# generate features using a different (less good) method for comparison
features_comp = cblast.features.extract_roioverlap_features(client, dataset, neuronlist)
hcluster_comp = cblast.cluster.hierarchical_cluster(features_comp)
cluster_comp, testmap_comp = hcluster_comp.get_partitions(5)

cdiff1, cdiff2 = cblast.cluster.report_diffs(cluster, cluster_comp, features, features_comp)

fetch batch 0


In [55]:
# Shows the neurons that are in the same cluster for first clustering but different
# clusters for the second clustering.  The largest difference is shown first.
cdiff1

[(0.8731916782800788, 858587718, 1228692168),
 (0.8595613972794941, 858587718, 1198330641),
 (0.7295269500240619, 1228692168, 1730608428),
 (0.7063270900621831, 416642425, 1011447819),
 (0.705963542320315, 633546217, 1011447819),
 (0.7041738634418965, 5813027103, 1011447819),
 (0.6964933881654253, 694920753, 1011447819),
 (0.6619630084385026, 634962055, 1011447819),
 (0.6524427934455339, 788794171, 1011447819),
 (0.6388632740295453, 1198330641, 1730608428),
 (0.5025790874597316, 918334668, 949710555),
 (0.4753802712432691, 948709216, 918334668),
 (0.4386549739679321, 949710555, 917647959),
 (0.4090479780870957, 948709216, 917647959)]

In [56]:
# Shows the neurons that are in the same cluster for second clustering but different
# clusters for the first clustering.  The largest difference is shown first.
cdiff2

[(0.8438347895115308, 1011447819, 978733459),
 (0.837580315247284, 1011447819, 1043825714),
 (0.8346482535035663, 1011447819, 541127846),
 (0.8307541722932323, 917647959, 1198330641),
 (0.8252239709850963, 918334668, 1198330641),
 (0.8244292887262823, 1011447819, 5813060726),
 (0.8186136105005515, 949710555, 1730608428),
 (0.8100812766729973, 948709216, 1730608428),
 (0.8092309776027645, 917647959, 1228692168),
 (0.808559488049184, 949710555, 858587718),
 (0.8060767050130995, 918334668, 1228692168),
 (0.7996841676895707, 948709216, 858587718)]